In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from scipy import stats
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode()
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_selection import chi2,f_classif,SelectKBest
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

plt.rc('figure',figsize=(20,11))

![](https://hivelife.com/wp-content/uploads/2019/09/Banner_TokyoTopRamen.jpg)


<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Introduction</h1>


<p style="text-align: center;"><span style='font-size: 22px; font-family: "Times New Roman", Times, serif;'>Ramen is without arguing a popular &apos;fast food&apos; not only in Japan but in many western countries as well but what differs between the many brands available what makes a certain type of ramen better than others, what features have the largest effect on the rating of the product, is it the company that produces the ramen? or maybe its the country of origin.&nbsp;</span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 22px;">We can see that there are many questions to be investigated, and using the data we have in this notebook; we can try and answer some of those questions.</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 22px;">Before diving into the EDA and modeling, let&apos;s first clearly define our goals in this kernel, what are some questions we will try to answer using the data at hand, and what we will try to predict.&nbsp;</span></span></p>
<p><span style="font-family: 'Times New Roman', Times, serif;"><br></span></p>
<h1 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><strong>&nbsp;Question We Will Investigate:</strong></span></h1>
<p><span style='font-family: "Times New Roman", Times, serif; font-size: 22px;'>1) What country produces the highest amount of ramen products&nbsp;</span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">2) Whats the country with the highest rating products&nbsp;</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">3) Does the product style affect the score directly? Does a certain kind of product style mean a lower score?</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">4) Is there a significant difference between brand average scores?</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style='font-family: "Times New Roman", Times, serif; font-size: 22px;'>5) Do the scores follow a normal distribution, or are the scores skewed? What&apos;s the brand that skews the distribution?&nbsp;</span></p>
<p><span style="font-family: 'Times New Roman', Times, serif;"><br></span></p>
<h1 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><strong>Feature Engineering Goals:</strong></span></h1>
<p><span style='font-family: "Times New Roman", Times, serif; font-size: 22px;'>1) Extract the meat type from the variety feature</span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">2) Extract the noodle type from the variety feature</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">3) Extract the number of words in the various feature</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">4) Extract the presence of the word &nbsp;&apos;Flavor&apos; from the variety feature</span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;">5) Extract the presence of the word &nbsp;&apos;Spicy&apos; to declare a product as spicy or not</span></span></p>
<p><span style="font-family: 'Times New Roman', Times, serif;"><br></span></p>
<p><span style="font-family: 'Times New Roman', Times, serif;"><br></span></p>
<h1 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><strong>&nbsp;Modeling And Prediction Goals:</strong></span></h1>
<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 22px;'>Our main goal as for prediction making will be to predict the ranking of a &apos;new&apos; ramen product based on the feature we may have before the rating like the variety feature, the brand, the country of origin, the new features will create, etc. so there is no risk of data leakage.</span></p>
<p style="text-align: center;"><span style="font-size: 22px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 22px;'>I think we are ready now to dive into our noodle bowl of information!</span></p>
<p><span style="font-family: 'Times New Roman', Times, serif;"><br></span></p>
<p><br></p>

In [ ]:
r_data = pd.read_csv('/kaggle/input/ramen-ratings/ramen-ratings.csv')
r_data.head(3)

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Data Preparation And Feature Engineering</h1>


In [ ]:
n_data = r_data.copy()
fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "heatmap",'rowspan':2},{"type": "table",'rowspan':2}],
           [None             ,None],
          ]
)

fig.add_trace(
    go.Heatmap(
        z=n_data.isna().T.astype(int),
        x=np.arange(0,len(n_data)),
        y=n_data.columns,
        showscale =False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Table(
    header=dict(values=list(['Feature Name','Number Of Missing']),
                fill_color='royalblue',
                font_color='white',
                font_size=13,
                align='left'),
    cells=dict(values=[n_data.columns,n_data.isna().sum().to_frame()[0]],
               fill_color='azure',
               font_size=14,
               align='left')),
    row=1, col=2
)




fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Proportion Of Missing Values",
)
fig.update_yaxes(title_text="Sentiment Strength")
fig.show()

In [ ]:
r_data.Style.fillna(r_data.Style.mode()[0],inplace=True)

<h2 style="text-align: center;">Only 2 values are missing from the &apos;Style&apos; feature. To place those values, we will use the mode, as for the &apos;Top Ten&apos; feature, it has more than 95% of values missing, I will not rely on this feature during my analysis and prediction.</h2>

In [ ]:
Style_dict = {r_data.Style.unique()[i]: i+1 for i in range(0,len(r_data.Style.unique()))}
Brand_dict = {r_data.Brand.unique()[i]: i+1 for i in range(0,len(r_data.Brand.unique()))}
Country_dict = {r_data.Country.unique()[i]: i+1 for i in range(0,len(r_data.Country.unique()))}

org_data = r_data.copy()
r_data.Style.replace(Style_dict,inplace=True)
r_data.Brand.replace(Brand_dict,inplace=True)
r_data.Country.replace(Country_dict,inplace=True)
r_data = r_data[r_data['Stars'] != 'Unrated']
r_data.head(3)

In [ ]:
#Variety Feature Extraction

#number of words extraction
def number_of_words(sir):
    return len(sir.split(' '))
r_data['Number_Of_Words'] = r_data.Variety.apply(number_of_words)

meat_types = ['chicken','beef','duck','pork','shrimp','turkey']
noodle_types = ['udon','soba','ramen','egg','shirataki','hokkien','noodles']

def contain_flavor(sir):
    stn = sir.lower()
    return 1 if 'flavor' in stn else 0

def meat_scanner(sir):
    stn = sir.lower()
    for meat in meat_types:
        if meat in stn:
            return meat
    return 'unspecified'

def noodle_scanner(sir):
    stn = sir.lower()
    for noodle in noodle_types:
        if noodle in stn:
            return noodle
    return 'unspecified'
    
    
def is_spicy(sir):
    stn = sir.lower()
    spc = ['spicy','hot','flaming','chili']
    for t in spc:
        if t in stn:
            return 1
    return 0

r_data['Contains_Flavor'] = r_data.Variety.apply(contain_flavor)
r_data['Meat_Type'] = r_data.Variety.apply(meat_scanner)
r_data['Noodle_Type'] = r_data.Variety.apply(noodle_scanner)
r_data['Is_Spicy'] = r_data.Variety.apply(is_spicy)

meat_type_dict = {r_data['Meat_Type'].unique()[i]: i+1 for i in range(0,len(r_data['Meat_Type'].unique()))}
noodle_type_dict = {r_data['Noodle_Type'].unique()[i]: i+1 for i in range(0,len(r_data['Noodle_Type'].unique()))}

r_data['Meat_Type'].replace(meat_type_dict,inplace=True)
r_data['Noodle_Type'].replace(noodle_type_dict,inplace=True)



w_data = r_data[['Brand','Style','Country','Number_Of_Words','Contains_Flavor','Meat_Type',
                'Noodle_Type','Is_Spicy','Stars']].copy()

w_data.head(10)

<h2 style="text-align: center;">We created all the features we aimed to create as well as encoding all the categorical features (later we will divide some of our features into &apos;dummy&apos; features depending on what we find out in our EDA) </h2>
<p><br></p>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Exploratory Data Analysis</h1>


In [ ]:
Brand_dict = {key : val for val,key in Brand_dict.items()}
Country_dict = {key : val for val,key in Country_dict.items()}
Style_dict = {key : val for val,key in Style_dict.items()}
noodle_type_dict= {key : val for val,key in noodle_type_dict.items()}
meat_type_dict =  {key : val for val,key in meat_type_dict.items()}

In [ ]:
top_25_brands =w_data['Brand'].value_counts().sort_values(ascending=False)[:25]
ax = sns.countplot(w_data[w_data['Brand'].isin(top_25_brands.index)]['Brand'],order = top_25_brands.index,
                  palette='mako')


ax.set_xticklabels([Brand_dict[int(ax.get_xticklabels()[i].get_text())] for i in range(0,25)],rotation=90)

ax.patches[0].set_fc('r')
ax.patches[1].set_fc((0.7,0.0,0.0))
ax.patches[2].set_fc((0.5,0.0,0.0))
ax.set_title('Distribution of the number of prodcuts made by different brands [Top 25]',fontsize=18)
plt.show()


<h2 style="text-align: center;">We see that that in our data there is a company which makes the most of the product,&apos; Nissin&apos; dominates our sample population of products having more than 350 different products in the dataset we see that if we look at the next 2 leading companies, they are fairly close to each other in the number of products they sale. As we go down our list, we see that the number of products decreases in a fairly balanced slope, meaning except &apos;Nissin,&apos; which dominates all the other companies, does not have as sharp a difference compared to &apos;Nissan.&apos;</h2>
<p><br></p>

In [ ]:
w_data.Stars = w_data.Stars.astype('float64')
pivot = w_data.groupby(by='Country').sum()
pivot = pivot.sort_values(by='Stars',ascending=False)
top_25_country_by_score = pivot[:25]

ax = sns.barplot(pivot.index,pivot.Stars,order = top_25_country_by_score.index,palette='mako')
ax.set_xticklabels([Country_dict[int(ax.get_xticklabels()[i].get_text())] for i in range(0,25)],rotation=90)
ax.patches[0].set_fc('r')
ax.patches[1].set_fc((0.7,0.0,0.0))
ax.patches[2].set_fc((0.5,0.0,0.0))
ax.set_title('Distribution of prodcuts made by different countries [Top 25]',fontsize=18)

plt.show()

<h2 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"> &apos;Japan,&apos; &apos;South Korea&apos; and &apos;USA&apos; are the countries with the highest amount of stars meaning they produce the most products and most of their products are fairly-highly rated.</span></h2>
<p><br></p>

In [ ]:
for t in Style_dict.keys():
    ax = sns.distplot(w_data[w_data['Style']==t]['Stars'],hist=False,label = Style_dict[t],kde_kws={'lw':4} )
plt.legend(prop={'size':20})
ax.set_title('Distributions of different ramen style rankings ',fontsize=18)

plt.show()

<h2 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"> We can see that most ramen style rankings are bimodal distributed between 3/4 and 5 except &apos;Box&apos; style, which is bimodally distributed between 1 and 5, meaning most of the &apos;Boxed&apos; ramen is highly rated with 4.2-4.4 stars on average. We also see that our data is negatively skewed, meaning most of the ramen products are rated with 3+ stars</span></h2>
<p><br></p>

In [ ]:
pivot = w_data.groupby(by='Style').mean()
pivot = pivot.sort_values(by='Stars',ascending=False)


ax = sns.barplot(x=pivot.index,y=pivot['Stars'],order=pivot.index,palette='mako')
ax.set_xticklabels([Style_dict[int(ax.get_xticklabels()[i].get_text())] for i in range(0,7)],rotation=90)
ax.patches[0].set_fc('r')
ax.patches[1].set_fc((0.7,0.0,0.0))
ax.patches[2].set_fc((0.5,0.0,0.0))
ax.set_title('Distribution of average rankings across different styles',fontsize=18)

plt.show()

In [ ]:
ax = sns.boxplot(x='Style',y='Stars',hue='Meat_Type',data=w_data)
l = plt.legend()
for i in range(0,6):
    l.get_texts()[i].set_text(list(meat_type_dict.values())[i])

    
ax.set_title('Distribution of rankings of different meat types across different styles',fontsize=18)    
plt.show()

### At most styles pork and beef are usually rated higher compared to chicken and shrimp

In [ ]:
labels = {x+1:list(noodle_type_dict.values())[x] for x in range(0,len(noodle_type_dict.values()))}
tmp = w_data.copy()
tmp.Noodle_Type = tmp.Noodle_Type.replace(labels)
ex.box(tmp,x='Style',y='Stars',color='Noodle_Type',title='Distribution of rankings of different noodle types across different styles',height=800,
      labels=labels)



### Udon And Ramen Usually Get Higher Rankings 

In [ ]:
ax = sns.distplot(w_data[w_data['Is_Spicy']==1]['Stars'],hist=False,label = "Spicy",kde_kws={'lw':4} )
ax = sns.distplot(w_data[w_data['Is_Spicy']==0]['Stars'],hist=False,label = "Not Spicy",kde_kws={'lw':4} )


plt.legend(prop={'size':20})

ax.set_title('Distribution of stars in spicy and not spicy products',fontsize=18)    
plt.show()

In [ ]:
ax = sns.distplot(w_data[w_data['Contains_Flavor']==1]['Stars'],hist=False,label = "Flavor In Var",kde_kws={'lw':4} )
ax = sns.distplot(w_data[w_data['Contains_Flavor']==0]['Stars'],hist=False,label = "Flavor Not In Var",kde_kws={'lw':4} )

plt.legend(prop={'size':20})


ax.set_title('Distribution of stars in labels containg the word Flavor and not',fontsize=18)    
plt.show()

In [ ]:
ax = sns.jointplot(w_data['Stars'],w_data['Number_Of_Words'],kind='kde',cmap='mako',height=14)


In [ ]:
meats = pd.get_dummies(w_data['Meat_Type'])
meats = meats.rename(meat_type_dict,axis=1)
w_data = w_data.join(meats[1:])
w_data.drop('Meat_Type',axis=1,inplace=True)


In [ ]:
style_ohe = pd.get_dummies(org_data.Style).drop(columns=['Tray'])
w_data = pd.concat([w_data,style_ohe],axis=1)

corrs = w_data.corr('pearson')

fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=('Perason Correaltion',  'Spearman Correaltion'))
colorscale=[[0.0, "rgb(165,0,38)"],
                [0.1111111111111111, "rgb(215,48,39)"],
                [0.2222222222222222, "rgb(244,109,67)"],
                [0.3333333333333333, "rgb(253,174,97)"],
                [0.4444444444444444, "rgb(254,224,144)"],
                [0.5555555555555556, "rgb(224,243,248)"],
                [0.6666666666666666, "rgb(171,217,233)"],
                [0.7777777777777778, "rgb(116,173,209)"],
                [0.8888888888888888, "rgb(69,117,180)"],
                [1.0, "rgb(49,54,149)"]]

sd_data = w_data.copy()

s_val =sd_data.corr('pearson')
s_idx = s_val.index
s_col = s_val.columns
s_val = s_val.values
fig.add_trace(
    go.Heatmap(x=s_col,y=s_idx,z=s_val,name='pearson',showscale=False,xgap=1,ygap=1,colorscale=colorscale),
    row=1, col=1
)


s_val =sd_data.corr('spearman')
s_idx = s_val.index
s_col = s_val.columns
s_val = s_val.values
fig.add_trace(
    go.Heatmap(x=s_col,y=s_idx,z=s_val,xgap=1,ygap=1,colorscale=colorscale),
    row=2, col=1
)

fig.update_layout(height=700, width=900, title_text="Correaltions Between Our Numeric Features")
fig.show()

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Model Selection And Evaluation</h1>


In [ ]:
selector = SelectKBest(f_classif,k=5)
w_data = w_data.fillna(0)
cols = [w_data.columns[i] for i in range(0,len(w_data.columns)) if w_data.columns[i] != 'Stars']
X = selector.fit_transform(w_data[cols],w_data['Stars'])

selected_columns = [cols[i] for i in range(0,len(cols)) if selector.get_support()[i] == True]
selected_columns

x_train,x_test,y_train,y_test = train_test_split(X,w_data['Stars'])



In [ ]:

#Linear Regression
LR_pipe = Pipeline(steps=[('scale',StandardScaler()),('model',LinearRegression())])
LR_cv_scores = -1*cross_val_score(LR_pipe,x_train,y_train,cv=5,scoring='neg_mean_squared_error')

LR_pipe.fit(x_train,y_train)
LR_predictions = LR_pipe.predict(x_test)
LR_score = np.sqrt(mean_squared_error(LR_predictions,y_test))

#RandomForest
RF_pipe = Pipeline(steps=[('scale',StandardScaler()),('model',RandomForestRegressor(random_state=42,
                                                                                    n_estimators=60,max_leaf_nodes=22))])
RF_cv_scores = -1*cross_val_score(RF_pipe,x_train,y_train,cv=5,scoring='neg_mean_squared_error')

RF_pipe.fit(x_train,y_train)
RF_predictions = RF_pipe.predict(x_test)
RF_score = np.sqrt(mean_squared_error(RF_predictions,y_test))


#AdaBoost
ADA_pipe = Pipeline(steps=[('scale',StandardScaler()),('model',AdaBoostRegressor(random_state=42,
                                                                                learning_rate=0.3,n_estimators=30))])
ADA_pipe.fit(x_train,y_train)
ADA_predictions = ADA_pipe.predict(x_test)
ada_cv_scores = -1*cross_val_score(ADA_pipe,x_train,y_train,cv=5,scoring='neg_mean_squared_error')
ada_score = np.sqrt(mean_squared_error(ADA_predictions,y_test))


#Knn Pipeline
KNN_pipe = Pipeline(steps=[('model',KNeighborsRegressor(n_neighbors=24))])
knn_cv_scores = -1*cross_val_score(KNN_pipe,x_train,y_train,cv=5,scoring='neg_mean_squared_error')
KNN_pipe.fit(X,w_data['Stars'])

#XGB 
xgb_model = XGBRegressor(n_estimators=300,learning_rate=0.03,random_state=42)
xgb_model.fit(x_train,y_train,early_stopping_rounds=7,eval_set=[(x_test[:5],y_test[:5])],verbose=False)
predictions = xgb_model.predict(x_test)
xgb_score = np.sqrt(mean_squared_error(predictions,y_test))

In [ ]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,subplot_titles=('Random Forest Cross Val Scores',
                                                                     'Linear Regression Cross Val Scores',
                                                                    'AdaBoost Cross Val Scores',
                                                                    'KNN Cross Val Scores'))

fig.add_trace(
    go.Scatter(x=np.arange(0,5),y=RF_cv_scores,mode='lines+markers',name='Random Forest'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(0,5),y=LR_cv_scores,mode='lines+markers',name='Linear Regression'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(0,5),y=ada_cv_scores,mode='lines+markers',name='AdaBoost'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(0,5),y=knn_cv_scores,mode='lines+markers',name='KNN'),
    row=4, col=1
)


#Fold Means
fig.add_shape(type="line",
    x0=0, y0=np.mean(RF_cv_scores), x1=5, y1=np.mean(RF_cv_scores),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y1'
)

fig.add_shape(type="line",
    x0=0, y0=np.mean(LR_cv_scores), x1=5, y1=np.mean(LR_cv_scores),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x2', 
        yref='y2'
)

fig.add_shape(type="line",
    x0=0, y0=np.mean(ada_cv_scores), x1=5, y1=np.mean(ada_cv_scores),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x3', 
        yref='y3'
)

fig.add_shape(type="line",
    x0=0, y0=np.mean(knn_cv_scores), x1=5, y1=np.mean(knn_cv_scores),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x4', 
        yref='y4'
)


fig.update_layout(height=700, width=900, title_text="Different Model 5 Fold Cross Validation")
fig.update_yaxes(title_text="RMSE")
fig.update_xaxes(title_text="Fold #")

fig.show()

In [ ]:
# Text Feature Extratction

NUMBER_OF_COMPONENTS=190

def preprocess_text(txt):
    ppt = txt.lower()
    ppt = ' '.join([word for word in re.findall(r'\w+',ppt) if len(word) >1 and word not in list(STOPWORDS)])
    return ppt
    
Variety = org_data['Variety'].apply(preprocess_text)

vectorizer = CountVectorizer()
sp_va_matrix = vectorizer.fit_transform(Variety)
t_SVD = TruncatedSVD(n_components = NUMBER_OF_COMPONENTS)
cp_matrix = t_SVD.fit_transform(sp_va_matrix)

cu_sum = np.cumsum(t_SVD.explained_variance_ratio_)

tr1 = go.Scatter(x=np.arange(0,len(cu_sum)),y=cu_sum,name='Explained Variance')
layout = dict(title=f'Explained Variance Using {NUMBER_OF_COMPONENTS} Components',xaxis_title='Number Of Components',yaxis_title='Explained Variance')
go.Figure(data=[tr1],layout=layout)

In [ ]:
CV_Model = LinearRegression()
CV_Model.fit(cp_matrix,w_data['Stars'])
Text_Based_Prediction = CV_Model.predict(cp_matrix)

X = np.append(X,Text_Based_Prediction.reshape((2580,1)),axis=1)

In [ ]:
xgb_model.fit(X,w_data['Stars'])
ADA_pipe.fit(X,w_data['Stars'])
LR_pipe.fit(X,w_data['Stars'])
RF_pipe.fit(X,w_data['Stars'])
KNN_pipe.fit(X,w_data['Stars'])

In [ ]:
glb = xgb_model.predict(X)
ada = ADA_pipe.predict(X)
rf = RF_pipe.predict(X)
lr = LR_pipe.predict(X)
knn = KNN_pipe.predict(X)

In [ ]:
ax=sns.lineplot(y=w_data['Stars'],x=np.arange(0,w_data.shape[0]),label='Actual')
ax=sns.lineplot(y=glb,x=np.arange(0,w_data.shape[0]),label='XGB Prediction')
ax=sns.lineplot(y=ada,x=np.arange(0,w_data.shape[0]),label='ADABoost Prediction')
ax=sns.lineplot(y=rf,x=np.arange(0,w_data.shape[0]),label='RandomForest Prediction')
ax=sns.lineplot(y=lr,x=np.arange(0,w_data.shape[0]),label='LinearRegression Prediction')
ax=sns.lineplot(y=lr,x=np.arange(0,w_data.shape[0]),label='KNN Prediction')



ax.set_title('Predicted values against real values',fontsize=20)
plt.show()

<h2 style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;">Our Model Blending Goes As Follows : $$Blended  =(XGB*0.6 + RandoForest*0.21 + KNN*0.2)$$</span></h2>
<p><br></p>

In [ ]:
fp = (glb*0.60+rf*0.21 + knn*0.2)

In [ ]:
print('RMSE Of Stacked Predictions: ',np.sqrt(mean_squared_error(w_data['Stars'],fp)))

In [ ]:
ax=sns.lineplot(y=w_data['Stars'],x=np.arange(0,w_data.shape[0]),label='Actual')
ax=sns.lineplot(y=fp,x=np.arange(0,w_data.shape[0]),label='Stacked Prediction')

ax.set_title('Final Prediction values against real values',fontsize=20)
plt.legend()
plt.show()

In [ ]:
output=pd.DataFrame({'Prediction':fp,'Actual':w_data['Stars']})
fig = make_subplots(
    rows=3, cols=2,subplot_titles=('','Actual','Predictions','Residuals'),
    vertical_spacing=0.09,
    specs=[[{"type": "table","rowspan": 3}     ,{"type": "scatter"}] ,
           [None                               ,{"type": "scatter"}]            ,           
           [None                               ,{"type": "scatter"}]                           
          ]
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Actual"])),
        y=output["Actual"],
        mode="markers",
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Prediction"])),
        y=output["Prediction"],
        mode="markers",
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Prediction"])),
        y=output["Prediction"]-output["Actual"],
        mode="markers",
    ),
    row=3, col=2
)

fig.add_trace(
    go.Table(
        header=dict(
            values=['Prediction','Actual'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[output[k].tolist() for k in output.columns],
            align = "left")
    ),
    row=1, col=1
)



fig.add_shape(type="line",
    x0=0, y0=(output["Prediction"]-output["Actual"]).mean(), x1=len(output["Prediction"]), y1=(output["Prediction"]-output["Actual"]).mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x3', 
        yref='y3'
)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Prediction Evaluation",
)

fig.show()

In [ ]:
plt.title('Stacked Model Residual Plot',fontsize=16)
sns.residplot(w_data['Stars'],fp)
plt.show()